#### Prerequisites

In [1]:
%%capture 

!pip install sagemaker==2.100.0
!pip install jedi==0.17

#### Imports 

In [2]:
from sagemaker.huggingface import HuggingFaceProcessor
from sagemaker.processing import ProcessingOutput
from sagemaker.processing import ProcessingInput
from sagemaker import get_execution_role
from sagemaker import Session
import sagemaker
import logging

##### Setup logging

In [3]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

##### Log versions of dependencies 

In [4]:
logger.info(f'[Using SageMaker version: {sagemaker.__version__}]')

[Using SageMaker version: 2.100.0]


#### Essentials

In [5]:
session = Session()
S3_BUCKET = session.default_bucket()
ROLE = get_execution_role()

S3_INPUT = 'data/covid_articles_clf_data.csv'
S3_OUTPUT = 'data/bert/processed-clf'
INSTANCE_TYPE = 'ml.g4dn.xlarge'
INSTANCE_COUNT = 1
PYTHON_VERSION = '1.6.0'
TRANSFORMERS_VERSION = '4.4.2'
BASE_JOB_NAME = 'hf-sagemaker-processor'
SOURCE_DIR = './src'
CODE = 'preprocess_data_clf_oob.py'

#### View processing script 

In [6]:
!pygmentize -v ./src/preprocess_data_clf_oob.py

from sklearn.metrics import precision_recall_fscore_support
from transformers import BertForSequenceClassification
from sklearn.metrics import accuracy_score
from transformers import TrainingArguments
from transformers import BertTokenizerFast
from transformers import pipeline
from transformers import Trainer
from datasets import load_dataset
from datasets import DatasetDict
import pandas as pd
import numpy as np
import pickle


# Setup logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.getLevelName('INFO'), 
                    handlers=[logging.StreamHandler(sys.stdout)], 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Log versions of dependencies
logger.info(f'[Using Transformers: {transformers.__version__}]')
logger.info(f'[Using Datasets: {datasets.__version__}]')
logger.info(f'[Using Sklearn: {sklearn.__version__}]')
logger.info(f'[Using Pandas: {pd.__version__}]')


# Essentials
# LOCAL_INPUT_PATH is mapped to S3

#### Create HuggingFace Processor

In [7]:
processor = HuggingFaceProcessor(role=ROLE, 
                                 instance_type=INSTANCE_TYPE, 
                                 instance_count=INSTANCE_COUNT, 
                                 transformers_version=TRANSFORMERS_VERSION, 
                                 pytorch_version=PYTHON_VERSION, 
                                 base_job_name=BASE_JOB_NAME)

#### Run SageMaker Processing job

In [8]:
%%time

processor.run(code=CODE, 
              source_dir=SOURCE_DIR,
              inputs=[ProcessingInput(input_name='article-headlines', 
                                      source=f's3://{S3_BUCKET}/{S3_INPUT}', 
                                      destination='/opt/ml/processing/input/data')],
              outputs=[ProcessingOutput(output_name='processed-data-oob-clf', 
                                        source='/opt/ml/processing/output', 
                                        destination=f's3://{S3_BUCKET}/{S3_OUTPUT}')])

Uploaded ./src to s3://sagemaker-us-east-1-119174016168/hf-sagemaker-processor-2022-09-01-21-26-15-423/source/sourcedir.tar.gz
runproc.sh uploaded to s3://sagemaker-us-east-1-119174016168/hf-sagemaker-processor-2022-09-01-21-26-15-423/source/runproc.sh
Creating processing-job with name hf-sagemaker-processor-2022-09-01-21-26-15-423
process request: {
    "ProcessingJobName": "hf-sagemaker-processor-2022-09-01-21-26-15-423",
    "ProcessingResources": {
        "ClusterConfig": {
            "InstanceType": "ml.g4dn.xlarge",
            "InstanceCount": 1,
            "VolumeSizeInGB": 30
        }
    },
    "AppSpecification": {
        "ImageUri": "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-training:1.6.0-transformers4.4.2-gpu-py36-cu110-ubuntu18.04",
        "ContainerEntrypoint": [
            "/bin/bash",
            "/opt/ml/processing/input/entrypoint/runproc.sh"
        ]
    },
    "RoleArn": "arn:aws:iam::119174016168:role/service-role/AmazonSageMaker-Ex


Job Name:  hf-sagemaker-processor-2022-09-01-21-26-15-423
Inputs:  [{'InputName': 'article-headlines', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-119174016168/data/covid_articles_clf_data.csv', 'LocalPath': '/opt/ml/processing/input/data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-119174016168/hf-sagemaker-processor-2022-09-01-21-26-15-423/source/sourcedir.tar.gz', 'LocalPath': '/opt/ml/processing/input/code/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'entrypoint', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-119174016168/hf-sagemaker-processor-2022-09-01-21-26-15-423/source/runproc.sh', 'LocalPath': '/opt/ml/processing/input/entrypoint', 'S3DataType': 'S3Prefix', 'S3Inpu

UnexpectedStatusException: Error for Processing job hf-sagemaker-processor-2022-09-01-21-26-15-423: Failed. Reason: AlgorithmError: See job logs for more information